#Generowanie klucza Prywatnego RSA:

In [11]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os

# Funkcja do generowania klucza prywatnego RSA z hasłem
def generate_encrypted_private_key(password, salt):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    # Utwórz kdf (Key Derivation Function) z hasłem i solą
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )

    # Wygeneruj klucz na podstawie hasła
    key = kdf.derive(password)

    # Ustaw algorytm szyfrowania AES-CBC z hasłem
    encryption_algorithm = serialization.BestAvailableEncryption(key)

    # Konwertuj klucz prywatny do formatu PEM i zaszyfruj go
    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=encryption_algorithm
    )

    return private_key_pem

# Hasło, które będzie używane do szyfrowania klucza
password = b"aaa"

# Losowa sól
salt = os.urandom(16)

# Wygeneruj zaszyfrowany klucz prywatny RSA
encrypted_private_key = generate_encrypted_private_key(password, salt)

# Zapis klucza prywatnego do pliku
with open("private_key.pem", "wb") as f:
    f.write(encrypted_private_key)


### Generowanie klucza publicznego

In [12]:
# Wczytaj klucz prywatny
with open("private_key.pem", "rb") as f:
    private_key_pem = f.read()

password = b"aaa"

# Odczytaj zaszyfrowany klucz prywatny
private_key = serialization.load_pem_private_key(
    private_key_pem,
    password=password,  # hasło użyte do zaszyfrowania klucza prywatnego
    backend=default_backend()
)

# Wygeneruj klucz publiczny
public_key = private_key.public_key()

# Konwersja klucza publicznego do formatu PEM
public_key_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Zapisz klucz publiczny do pliku
with open("public_key.pem", "wb") as f:
    f.write(public_key_pem)


ValueError: Bad decrypt. Incorrect password?